![banner](https://anopheles-genomic-surveillance.github.io/_images/banner.jpg)

***Training course in data analysis for genomic surveillance of African malaria vectors - Workshop 5***

---

# Module 1 - Xarray datasets

**Theme: Tools & technology**

This module introduces [xarray](https://docs.xarray.dev/en/stable/), a Python package for working with more complex scientific datasets, including datasets comprising multidimensional arrays with shared and labelled dimensions. Like other general-purpose scientific computing packages like NumPy and pandas, xarray is very useful for genomic data, but it is also used in a variety of other scientific fields. 

This module is intended to provide a gentle introduction to xarray for scientists and analysts coming from a malaria or genetics background. Please note that there is an excellent [xarray tutorial website](https://tutorial.xarray.dev/intro.html) and [xarray YouTube channel](https://www.youtube.com/channel/UCBlxVSA6xQXeb-i4GgTlO7g) with much more information on what xarray can do and how to use it. 

## Learning objectives

At the end of this module you will be able to:

* Explain why xarray is useful for scientific computing
* Explain what an xarray dataset is
* Access variables (arrays) in a dataset
* Use indexing to select data within a dataset
* Review how we use xarray for genomic data


## Lecture

### English

@@TODO

### Français

@@TODO

## Setup

Install and import the packages we'll need.

In [1]:
!pip install -q malariagen_data rioxarray

In [3]:
import malariagen_data
import xarray as xr
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = "retina"

Configure access to the MalariaGEN Ag3 data resource.

In [4]:
ag3 = malariagen_data.Ag3()
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0
Results cache           : None
Cohorts analysis        : 20220608
Species analysis        : aim_20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 6.0.0
Client location         : Iowa, US (Google Cloud)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

## Introduction

### Recap: multidimensional arrays

Previously in [workshop 4, module 1](../workshop-4/module-1-numpy) we introduced the concept of multidimensional arrays, and how they can be used to store some different types of scientific data. Let's recap two examples.

The first example is a grayscale image, which can be stored as a 2-dimensional array of numbers. Each element in the array represents a pixel, and the value of the number encodes the colour. One dimension of the array corresponds to the vertical position of the pixels within the image, and the other dimension of the array corresponds to the horizontal position of the pixels.

<img src="https://jalammar.github.io/images/numpy/numpy-grayscale-image.png" width="700"/>

The second example is a set of genotype calls obtained from sequencing some mosquitoes. These data can be stored as a 3-dimensional array, where one dimension of the array corresponds to positions (sites) within a reference genome, another dimension corresponds to the individual mosquitoes that were sequenced, and a third dimension corresponds to the number of genomes within each individual (mosquitoes are diploid, hence this dimension is of length 2).

<img src="https://storage.googleapis.com/vo_agam_release/reference/training-images/workshop-4/mosquito-genotype-array.png" width="650"/>

### Why xarray?

Xarray is designed for situations where you have more than one multidimensional data array that you need to manage and access. To illustrate a situation where this is useful, consider the following image, which shows data from a geostatistical model of insecticide-treated net (ITN) use in Africa:

<img src="https://storage.googleapis.com/vo_agam_release/reference/training-images/workshop-5/itn_use.png" width="650"/>

This model was created by the [Malaria Atlas Project](https://malariaatlas.org/), and the image above is a screenshot from their online [data explorer tool](https://malariaatlas.org/explorer/). The model predicts the degree of ITN use within 5km squares across sub-Saharan Africa, for each year between 2000 and 2020. For each 5km square and year, the model generates a numerical value between 0 and 1, where a value of 0 means no-one uses an ITN, and a value of 1 means everyone uses an ITN. Thus the model output is a 3-dimensional array, with two spatial dimensions corresponding to longitude and latitude, and a third dimension corresponding to year. In the image above, the model output for 2019 is shown by colouring each grid square using a colour map ranging from red (0) to green (1).

These data are a good use case for xarray, because in addition to overall ITN use, the model also breaks this down into several other data variables. In particular, the model outputs ITN access (what fraction of people have access to a net), ITN use rate (what fraction of people with access to a net choose to use it), and ITNs per capita (how many ITNs have been supplied to the population). The model is described in [Bertozzi-Villa et al. (2021)](https://doi.org/10.1038/s41467-021-23707-7), and [Figure 3](https://www.nature.com/articles/s41467-021-23707-7/figures/3) provides a nice visualising of these data variables for the year 2020. 

Thus, in the data outputs generated by this model, there are:

* **Several multidimensional arrays**. E.g., ITN access, ITN use rate, ITN per capita. These are called "data variables" in xarray.
* **Dimensions are shared**. E.g., all data variables are 3-dimensional, using the same grid of 5km squares, and the same range of years.
* **Dimensions correspond to some kind of coordinate system**. E.g., the spatial dimensions correspond to latitude and longitude values, and the temporal dimension corresponds to year values between 2000 and 2020. These are called "coordinate variables" in xarray.

Let's now look at how xarray can be used to represent the data from this model.

## Anatomy of an xarray dataset

In [ ]:
def load_itn_metrics():

    # need to combined data from multiple tifs, requires rioxarray
    datasets = []
    years = np.arange(2000, 2021)
    file_paths = {
        "itn_access_mean": "https://storage.googleapis.com/malariagen-reference-data-us/malariaatlas/2020_Africa_ITN_Access_mean.zip/ITN_{year}_access_mean.tif",
        "itn_per_capita_mean": "https://storage.googleapis.com/malariagen-reference-data-us/malariaatlas/2020_Africa_ITN_Percapita_Nets_mean.zip/ITN_{year}_percapita_nets_mean.tif",
        "itn_use_mean": "https://storage.googleapis.com/malariagen-reference-data-us/malariaatlas/2020_Africa_ITN_Use_mean.zip/ITN_{year}_use_mean.tif",
        "itn_use_rate_mean": "https://storage.googleapis.com/malariagen-reference-data-us/malariaatlas/2020_Africa_ITN_Use_Rate_mean.zip/ITN_{year}_use_rate_mean.tif",
    }
    for variable_name, file_path_template in file_paths.items():    
        ds = (
            xr.open_mfdataset(
                paths=[file_path_template.format(year=year) for year in range(2000, 2021)],
                engine="rasterio",
                combine="nested",
                concat_dim="year",
            )
            .rename_dims({"x": "lon", "y": "lat"})
            .rename_vars({"x": "lon", "y": "lat", "band_data": variable_name})
            .isel(band=0)
            .drop_vars(["band", "spatial_ref"])
        )
        ds.coords["year"] = "year", years
        datasets.append(ds)

    # merge all datasets, assume already aligned
    ds_itn = xr.merge(
        datasets,
        compat="override",
        join="override",
    )

    # add metadata
    ds_itn.attrs["title"] = "Maps and metrics of insecticide-treated net access, use, and nets-per-capita in Africa from 2000-2020"
    ds_itn.attrs["creator"] = "The Malaria Atlas Project"
    ds_itn.attrs["references"] = "https://malariaatlas.org/research-project/metrics-of-insecticide-treated-nets-distribution/"

    return ds_itn


In [ ]:
ds_itn = load_itn_metrics()

In [ ]:
type(ds_itn)

In [ ]:
ds_itn

* Named dimensions
* Dimension sizes (lengths)
* Data variables
* Coordinate variables
* Attributes

In [ ]:
ds_itn.sel(year=2019)["itn_use_mean"].plot(vmin=0, vmax=1, cmap="RdYlGn");

## Accessing variables

In [ ]:
itn_use_mean = ds_itn["itn_use_mean"]

In [ ]:
type(itn_use_mean)

In [ ]:
itn_use_mean.ndim

In [ ]:
itn_use_mean.shape

In [ ]:
itn_use_mean.dtype

In [ ]:
itn_use_mean.dims

In [ ]:
itn_use_mean

In [ ]:
a = itn_use_mean.values

In [ ]:
type(a)

In [ ]:
longitude = ds_itn["lon"]

In [ ]:
type(longitude)

In [ ]:
longitude

In [ ]:
longitude.values

In [ ]:
year = ds_itn["year"]

In [ ]:
year

In [ ]:
year.values

## Selecting data with indexing

In [ ]:
ds_itn

### Positional indexing with `isel()`

In [ ]:
ds_itn_y0 = ds_itn.isel(year=0)
ds_itn_y0

In [ ]:
ds_itn_y02 = ds_itn.isel(year=slice(0, 2))
ds_itn_y02

### Label-based indexing with `sel()`

In [ ]:
ds_itn_2020 = ds_itn.sel(year=2020)
ds_itn_2020

In [ ]:
ds_itn_2020["itn_access_mean"].plot(vmin=0, vmax=1, cmap="YlGnBu");

In [ ]:
ds_itn_2020["itn_use_rate_mean"].plot(vmin=0, vmax=1, cmap="Spectral");

In [ ]:
ds_itn_2020["itn_per_capita_mean"].plot(vmin=0, vmax=1, cmap="YlOrBr");

In [ ]:
ds_itn_2018_2020 = ds_itn.sel(year=slice(2018, 2020))
ds_itn_2018_2020

## Uses of xarray for genomic data

### SNP calls

In [ ]:
ds_snp = ag3.snp_calls(
    region="3R"
)
ds_snp

In [ ]:
ds_snp.isel(samples=0)

In [ ]:
gt = ds_snp.isel(samples=0)["call_genotype"].values
gt

In [ ]:
ds_snp_ix = ds_snp.set_index(samples="sample_id")
ds_snp_ix

In [ ]:
ds_snp_ix.sel(samples="AR0047-C")

In [ ]:
gt = ds_snp_ix.sel(samples="AR0047-C")["call_genotype"].values
gt

### CNV calls

In [ ]:
ds_cnv = ag3.cnv_hmm(
    region="3R"
)
ds_cnv

In [ ]:
ds_cnv_ix = ds_cnv.set_index(samples="sample_id")
ds_cnv_ix

In [ ]:
cn = ds_cnv_ix.sel(samples="AR0047-C")["call_CN"].values
cn

### Allele frequencies

In [ ]:
cyp6aap_cnv_ds = ag3.gene_cnv_frequencies_advanced(
    region="2R:28,480,000-28,510,000",
    area_by="admin1_iso",
    period_by="year",
    variant_query="max_af > 0.05"
)
cyp6aap_cnv_ds

In [ ]:
ag3.plot_frequencies_interactive_map(
    cyp6aap_cnv_ds,
    title="Gene CNV frequencies, Cyp6aa/p locus"
)

## Well done!